In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import folium
%matplotlib inline

from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim
import requests

# import credentials
from credentials import CLIENT_CREDENTIALS

In [2]:
toronto_geo = pd.read_csv("data/toronto_geo.csv")
toronto_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [3]:
geolocator = Nominatim(user_agent="toronto_app")
location = geolocator.geocode("Toronto")
toronto_location = (location.latitude, location.longitude)

print(toronto_location)

(43.6534817, -79.3839347)


In [4]:
map_toronto = folium.Map(location=toronto_location, zoom_start=10)

for postcode, borough, neighborhood, lat, long in toronto_geo.values:
    label = folium.Popup(borough + ", " + neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

In [5]:
# get borough if contains 'toronto'
boroughs = toronto_geo["Borough"].unique()
toronto_boroughs = []

for borough_name in boroughs:
    if "toronto" in borough_name.lower():
        toronto_boroughs.append(borough_name)

In [6]:
toronto_boroughs

['Downtown Toronto',
 'East Toronto',
 'West Toronto',
 'East York/East Toronto',
 'Central Toronto',
 'Downtown Toronto Stn A',
 'East Toronto Business']

In [7]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_borough_geo = toronto_geo[toronto_geo['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
toronto_borough_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [8]:
map_toronto = folium.Map(location=toronto_location, zoom_start=10)

for postcode, borough, neighborhood, lat, long in toronto_borough_geo.values:
    label = folium.Popup(borough + ", " + neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

In [9]:
# get 100 top venues in 500 meter radius
max_radius = 500
limit = 100

top_venues = []

for postcode, borough, neighborhood, lat, long in toronto_borough_geo.values:
    base_url = "https://api.foursquare.com/v2/venues/explore"
    base_url += "?client_id=" + CLIENT_CREDENTIALS["CLIENT_ID"]
    base_url += "&client_secret=" + CLIENT_CREDENTIALS["CLIENT_SECRET"]
    base_url += "&v=" + "20180605"
    base_url += "&ll=" + str(lat) + "," + str(long)
    base_url += "&radius=" + str(max_radius)
    base_url += "&limit=" + str(limit)
    
    results = requests.get(base_url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        top_venues.append({
            "postal_code": postcode,
            "borough": borough,
            "neighborhood": neighborhood,
            "latitude": lat,
            "longitude": long,
            "venue_name": venue['venue']['name'],
            "venue_latitude": venue['venue']['location']['lat'],
            "venue_longitude": venue['venue']['location']['lng'],
            "venue_category": venue['venue']['categories'][0]['name']
        })

In [10]:
top_venues[2]

{'postal_code': 'M5A',
 'borough': 'Downtown Toronto',
 'neighborhood': 'Regent Park, Harbourfront',
 'latitude': 43.6542599,
 'longitude': -79.3606359,
 'venue_name': 'Cooper Koo Family YMCA',
 'venue_latitude': 43.65324910177244,
 'venue_longitude': -79.35800826343677,
 'venue_category': 'Distribution Center'}

In [11]:
# convert top venues to dataframe
venues_data = pd.DataFrame(top_venues)
venues_data.head()

,postal_code,borough,neighborhood,latitude,longitude,venue_name,venue_latitude,venue_longitude,venue_category
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [12]:
# see top venues per postalcode
venues_data.groupby(["postal_code", "borough", "neighborhood"]).count()

,,,latitude,longitude,venue_name,venue_latitude,venue_longitude,venue_category
postal_code,borough,neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4J,East York/East Toronto,The Danforth East,2,2,2,2,2,2
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",18,18,18,18,18,18
M4M,East Toronto,Studio District,36,36,36,36,36,36
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,10,10,10,10,10,10
M4R,Central Toronto,North Toronto West,19,19,19,19,19,19
M4S,Central Toronto,Davisville,35,35,35,35,35,35


In [13]:
venues_data['venue_category'].value_counts()

Coffee Shop             146
Café                     84
Restaurant               50
Italian Restaurant       38
Hotel                    36
                       ... 
Frozen Yogurt Shop        1
Boat or Ferry             1
Massage Studio            1
Other Great Outdoors      1
Airport Food Court        1
Name: venue_category, Length: 229, dtype: int64

## Neighborhood Clustering using latitude and longitude

In [14]:
toronto_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [15]:
kmeans = KMeans(n_clusters=5, random_state=0)
kmeans.fit(toronto_geo[["Latitude", "Longitude"]])

KMeans(n_clusters=5, random_state=0)

In [16]:
clusters = kmeans.predict(toronto_geo[["Latitude", "Longitude"]])

In [17]:
toronto_geo['clusters'] = clusters
toronto_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,clusters
0,M3A,North York,Parkwoods,43.753259,-79.329656,4
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2


In [18]:
cluster_color = [
    'red',
    'green',
    'blue',
    'yellow',
    'cyan'
]

#### Map clusters 

In [19]:
map_toronto = folium.Map(location=toronto_location, zoom_start=10)

for postcode, borough, neighborhood, lat, long, cluster in toronto_geo.values:
    label = folium.Popup(borough + ", " + neighborhood + ", Cluster " + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=cluster_color[cluster],
        fill=True,
        fill_color=cluster_color[cluster],
        fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

In [20]:
for i in range(5):
    # print borough and neighborhood for each cluster
    bn = toronto_geo[toronto_geo['clusters']==i][["Borough", "Neighborhood"]]
    print("cluster", i, ": ", bn.values)

cluster 0 :  [['Scarborough' 'Malvern, Rouge']
 ['Scarborough' 'Rouge Hill, Port Union, Highland Creek']
 ['Scarborough' 'Guildwood, Morningside, West Hill']
 ['Scarborough' 'Woburn']
 ['Scarborough' 'Cedarbrae']
 ['Scarborough' 'Scarborough Village']
 ['Scarborough' 'Kennedy Park, Ionview, East Birchmount Park']
 ['Scarborough' 'Cliffside, Cliffcrest, Scarborough Village West']
 ['Scarborough' 'Dorset Park, Wexford Heights, Scarborough Town Centre']
 ['Scarborough' 'Wexford, Maryvale']
 ['Scarborough' 'Agincourt']
 ['Scarborough' "Clarks Corners, Tam O'Shanter, Sullivan"]
 ['Scarborough'
  "Milliken, Agincourt North, Steeles East, L'Amoreaux East"]
 ['Scarborough' "Steeles West, L'Amoreaux West"]
 ['Scarborough' 'Upper Rouge']]
cluster 1 :  [['Etobicoke' 'Islington Avenue']
 ['Etobicoke'
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale']
 ['Etobicoke'
  'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood']
 ['North York' 'Downsview West']
 ['North Y